In [1]:
// Use TestContainers to start a postgres database
@file:DependsOn("org.testcontainers:junit-jupiter:1.21.3")
@file:DependsOn("org.testcontainers:postgresql:1.20.0")

import org.testcontainers.containers.PostgreSQLContainer
import org.testcontainers.utility.DockerImageName
import org.testcontainers.utility.MountableFile


// Start the container with some preconfigured test data
val postgres = PostgreSQLContainer(DockerImageName.parse("postgres:16-alpine"))
    .withCopyToContainer(
        MountableFile.forHostPath("../src/test/resources/init_postgres.sql"),
        "/docker-entrypoint-initdb.d/init.sql"
    )
postgres.start()
DISPLAY("Is running: ${postgres.isRunning}")
DISPLAY("URL: ${postgres.jdbcUrl}")

Is running: true

URL: jdbc:postgresql://localhost:54350/test?loggerLevel=OFF

In [2]:
import org.jetbrains.exposed.v1.core.plus

val dataSrc = createDataSource(
    jdbcUrl = postgres.jdbcUrl,
    username = "test",
    password = "test"
)
dataSrc

HikariDataSource (HikariPool-1)

In [3]:
import org.jetbrains.exposed.v1.core.Table
import org.jetbrains.exposed.v1.core.dao.id.IntIdTable
import org.jetbrains.exposed.v1.jdbc.Database

// Define schema using Exposed
object Users : IntIdTable("users") {
    val name = text("name")
}

val db = Database.connect(dataSrc)
db

ExposedDatabase[1464461737](null)

In [4]:
import org.jetbrains.exposed.v1.core.ResultRow
import org.jetbrains.exposed.v1.jdbc.insert
import org.jetbrains.exposed.v1.jdbc.selectAll
import org.jetbrains.exposed.v1.jdbc.transactions.transaction

// Read using Exposed schema API
transaction {
    Users.selectAll().forEach { row: ResultRow ->
        println("${row[Users.id]}: ${row[Users.name]}")
    }
}

1: Alice
2: Bob
3: Charlie


In [5]:
// Write using Exposed schema API
transaction {
    Users.insert { row ->
        row[Users.name] = "Dennis"
    }
    Users.selectAll().forEach { row: ResultRow ->
        println("${row[Users.id]}: ${row[Users.name]}")
    }
}

1: Alice
2: Bob
3: Charlie
4: Dennis


In [6]:
import org.jetbrains.exposed.v1.core.dao.id.EntityID
import org.jetbrains.exposed.v1.dao.IntEntity
import org.jetbrains.exposed.v1.dao.IntEntityClass

// Define Exposed DAO Object
class UserEntity(id: EntityID<Int>) : IntEntity(id) {
    companion object : IntEntityClass<UserEntity>(Users)

    var name by Users.name

    override fun toString(): String {
        return "Task(id=$id, name=$name)"
    }
}

In [7]:
// Read using Exposed DAO API
transaction {
    UserEntity.all().toList().joinToString("\n")
}

Task(id=1, name=Alice)
Task(id=2, name=Bob)
Task(id=3, name=Charlie)
Task(id=4, name=Dennis)

In [8]:
// Write using Exposed DAO API
transaction {
    UserEntity.new {
        name = "Ellie"
    }
    UserEntity.all().toList().joinToString("\n")
}

Task(id=1, name=Alice)
Task(id=2, name=Bob)
Task(id=3, name=Charlie)
Task(id=4, name=Dennis)
Task(id=5, name=Ellie)

In [9]:
// Clear the table again
import org.jetbrains.exposed.v1.jdbc.deleteAll

transaction {
    Users.deleteAll()
    UserEntity.count()
}

0

In [10]:
postgres.stop()